In [1]:
cuda = True
train_batch_size = 32
test_batch_size = 124
best_loss = float("inf")
best_epoch = -1
dataset_path = './cifar10'
gsync_save = True
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torchvision
from torchvision import datasets, transforms

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

class StridedC(nn.Module):
    def __init__(self, input_size, n_classes=10, **kwargs):
        super(StridedC, self).__init__()
        self.conv1 = nn.Conv2d(input_size, 96, 3, padding=1)
        self.conv2 = nn.Conv2d(96, 96, 3, padding=1, stride=2) 
        self.conv3 = nn.Conv2d(96, 192, 3, padding=1) 
        self.conv4 = nn.Conv2d(192, 192, 3, stride=2, padding=1) 
        self.conv5 = nn.Conv2d(192, 192, 3, padding=1) 
        self.conv6 = nn.Conv2d(192, 192, 1)
        self.class_conv = nn.Conv2d(192, n_classes, 1)


    def forward(self, x):
        x_drop = F.dropout(x, .2)
        conv1_out = F.relu(self.conv1(x_drop))
        conv2_out = F.relu(self.conv2(conv1_out))
        conv2_out_drop = F.dropout(conv2_out, .5)
        conv3_out = F.relu(self.conv3(conv2_out_drop))
        conv4_out = F.relu(self.conv4(conv3_out))
        conv4_out_drop = F.dropout(conv4_out, .5)
        conv5_out = F.relu(self.conv5(conv4_out_drop))
        conv6_out = F.relu(self.conv6(conv5_out))
        class_out = F.relu(self.class_conv(conv6_out))
        pool_out = F.adaptive_avg_pool2d(class_out, 1)
        pool_out.squeeze_(-1)
        pool_out.squeeze_(-1)
        return pool_out

trial=StridedC(3)
print(trial)

summary(trial.cuda(),(3,32,32))

StridedC(
  (conv1): Conv2d(3, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(96, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv3): Conv2d(96, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(192, 192, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv5): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1))
  (class_conv): Conv2d(192, 10, kernel_size=(1, 1), stride=(1, 1))
)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 32, 32]           2,688
            Conv2d-2           [-1, 96, 16, 16]          83,040
            Conv2d-3          [-1, 192, 16, 16]         166,080
            Conv2d-4            [-1, 192, 8, 8]         331,968
            Conv2d-5            [-1, 192, 8, 8]         331,968
            Conv2d-6

In [0]:
cuda =torch.cuda.is_available()
trainset = datasets.CIFAR10(root=dataset_path, train=True, download=True)
train_mean = trainset.data.mean(axis=(0,1,2))/255  # [0.49139968  0.48215841  0.44653091]
train_std = trainset.data.std(axis=(0,1,2))/255  # [0.24703223  0.24348513  0.26158784]
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std),
])
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
train_loader = torch.utils.data.DataLoader(datasets.CIFAR10(
    root=dataset_path, train=True, download=True,
    transform=transform_train),
    batch_size=train_batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(root=dataset_path, train=False, download=True,
    transform=transform_test),
    batch_size=test_batch_size, shuffle=False, **kwargs)

Extracting ./cifar10/cifar-10-python.tar.gz to ./cifar10
Files already downloaded and verified
Files already downloaded and verified


In [0]:


model = StridedC(3)
if cuda:
    model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay = 0.001)
scheduler = optim.lr_scheduler.MultiStepLR(
    optimizer, milestones=[200, 250, 300], gamma=0.1)



In [0]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data.item()))
            
def test(epoch, best_loss, best_epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).data.item()
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(
        '\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset), 100. * correct /
            len(test_loader.dataset)))
    
    if test_loss < best_loss:
        best_epoch = epoch
        best_loss = test_loss
        model_save_name = 'StridedC_0_005.pth'
        path = F"/content/gdrive/My Drive/{model_save_name}"
        torch.save(model.state_dict(), path)
        print("new best")
        print(best_epoch)
        #torch.save(model, "best.pt")
        # if gsync_save:
        #     gsync.update_file_to_folder("best.pt")
    return best_loss, best_epoch

In [0]:
for epoch in range(350):
    train(epoch)
    scheduler.step()
    best_loss, best_epoch = test(epoch, best_loss, best_epoch)

#model_save_name = 'StridedC_0_005.pth'
#path = F"/content/gdrive/My Drive/{model_save_name}"
#torch.save(model.state_dict(), path)

Streaming output truncated to the last 5000 lines.
Train Epoch: 89 [6400/50000 (13%)]	Loss: 0.665382
Train Epoch: 89 [9600/50000 (19%)]	Loss: 0.553036
Train Epoch: 89 [12800/50000 (26%)]	Loss: 0.459918
Train Epoch: 89 [16000/50000 (32%)]	Loss: 0.710388
Train Epoch: 89 [19200/50000 (38%)]	Loss: 0.884181
Train Epoch: 89 [22400/50000 (45%)]	Loss: 0.608512
Train Epoch: 89 [25600/50000 (51%)]	Loss: 0.507983
Train Epoch: 89 [28800/50000 (58%)]	Loss: 0.500275
Train Epoch: 89 [32000/50000 (64%)]	Loss: 0.550780
Train Epoch: 89 [35200/50000 (70%)]	Loss: 0.724856
Train Epoch: 89 [38400/50000 (77%)]	Loss: 0.761442
Train Epoch: 89 [41600/50000 (83%)]	Loss: 0.602247
Train Epoch: 89 [44800/50000 (90%)]	Loss: 0.812181
Train Epoch: 89 [48000/50000 (96%)]	Loss: 0.925930

Test set: Average loss: 0.0057, Accuracy: 7592/10000 (76%)

new best
89
Train Epoch: 90 [0/50000 (0%)]	Loss: 0.745315
Train Epoch: 90 [3200/50000 (6%)]	Loss: 1.082808
Train Epoch: 90 [6400/50000 (13%)]	Loss: 0.675328
Train Epoch: 90 [96